In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Andrew-PROJECT/titanic

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Andrew-PROJECT/titanic


# 1. Load Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

In [3]:
train_data = pd.read_csv('train_for_pipeline.csv')
X_test = pd.read_csv('test_for_pipeline.csv')

X_train = train_data.drop("Survived", axis = 1)
y_train = train_data["Survived"]

df_test = pd.read_csv('test.csv')

# 2. Logistic Regression model

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection  import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

cv = StratifiedKFold(n_splits=3, shuffle=False, random_state=0)

logreg = LogisticRegression()
imp = SimpleImputer(strategy = 'mean')
#scaler = StandardScaler()
scaler = MinMaxScaler()
steps = [('imp', imp), ("scaler", scaler),('logreg', logreg)]
pipe = Pipeline(steps)

param_grid = [
    {'logreg__penalty': ['l1'], 'logreg__solver': ['liblinear'],
         'logreg__C':[0.1, 0.2, 0.5, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 2, 2.5, 50], 'logreg__max_iter':[100, 200] },
    {'logreg__C':[0.1, 0.2, 0.5, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 2, 2.5, 50], 'logreg__max_iter':[100, 200]}
]
logreg_gridcv = GridSearchCV(pipe, param_grid = param_grid, cv=cv, scoring="accuracy")
logreg_gridcv.fit(X_train, y_train) 

In [21]:
logreg_gridcv.best_score_, logreg_gridcv.best_params_

(0.7957351290684623,
 {'logreg__C': 0.5,
  'logreg__max_iter': 100,
  'logreg__penalty': 'l1',
  'logreg__solver': 'liblinear'})

In [22]:
logreg_gridcv.score(X_train,y_train)

0.8013468013468014

In [46]:
# Enregistrement
pickle.dump(logreg_gridcv, open('logreg_cv.sav', 'wb'))

In [ ]:
results = pd.DataFrame(logreg_gridcv.cv_results_)
print(results[['params',  'mean_test_score']])


In [39]:
y_pred = logreg_gridcv.predict(X_test)
df_result = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_pred})
df_result.to_csv('reluts_logreg.csv', index = False)

# 3. Linear SVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer



imp_svc = SimpleImputer(strategy = 'mean')
#scaler_svc = StandardScaler()
scaler_svc = MinMaxScaler()
svc = LinearSVC()

steps_svc = [('imp_svc', imp_svc), ('scaler', scaler_svc), ('svc', svc)]
pipe_svc = Pipeline(steps_svc)

param_grid_svc = {'svc__C': [0.1, 0.15, 0.2, 0.3, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
svc_gridcv = GridSearchCV(pipe_svc, param_grid = param_grid_svc, cv=cv, scoring="accuracy")

svc_gridcv.fit(X_train, y_train)



In [52]:
svc_gridcv.best_params_, svc_gridcv.score(X_train, y_train)

({'svc__C': 7}, 0.8002244668911336)

In [54]:
y_pred = svc_gridcv.predict(X_test)
df_result = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_pred})
df_result.to_csv('reslut_svc.csv', index = False)

# 4. KNN model

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

imp_knn = SimpleImputer(strategy = 'mean')
scaler_knn = StandardScaler()
#scaler_knn = MinMaxScaler()
knn = KNeighborsClassifier()

steps_knn = [('imp', imp_knn), ('scaler', scaler_knn), ('knn', knn)]
pipe_knn = Pipeline(steps_knn)
param_grid_knn = {'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
knn_gridcv = GridSearchCV(pipe_knn, param_grid = param_grid_knn, cv=cv, scoring="accuracy")

knn_gridcv.fit(X_train, y_train)


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=False),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('imp',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='mean',
                                                      verbose=0)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='a

In [27]:
knn_gridcv.best_params_
knn_gridcv.best_score_

0.8103254769921436

In [28]:
knn_gridcv.score(X_train, y_train)

0.8698092031425365

In [29]:
y_pred = knn_gridcv.predict(X_test)

# 100. OUTPUT


In [30]:
df_test = pd.read_csv('test.csv')

In [31]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [37]:
df_result = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_pred})
df_result.to_csv('reluts_knn.csv', index = False)